### Frequency extraction

In [ ]:
%matplotlib inline
import soundfile as sf
import IPython.display as ipd
import numpy as np
import math as math
import matplotlib.pyplot as plt
import scipy
from librosa.display import specshow
import librosa

In [ ]:
VIBRATO_WAV = '../datasets/guitar/vibrato.wav'
OPENA_WAV = '../datasets/guitar/open-a.wav'
OPENE_HIGH_WAV = '../datasets/guitar/open-high-e.wav'
HARMONIC_WAV = '../datasets/guitar/12th-fret-harmonic.wav'
HOP_SIZE = 256
WIN_SIZE = 512

In [ ]:
def sinusoid(freq=440.0, dur=1.0, srate=44100.0, amp=1.0, phase = 0.0): 
    t = np.linspace(0,dur,int(srate*dur))
    data = amp * np.sin(2*np.pi*freq *t+phase)
    return data
def oscillator(freq=440.0, dur=1.0, srate=44100.0, amp=1.0, phase=0.0, wave_type='sinusoid'):
    
    if wave_type not in ['sinusoid', 'triangle', 'square']:
        return None
    
    if wave_type == 'sinusoid':
        return sinusoid(freq=freq, dur=dur, srate=srate, amp=amp, phase=phase)
    
    wave = sinusoid(freq=freq, dur=dur, amp=amp)
    
    for i in range(1,20):
        harmonic = 2*i+1
        sin = sinusoid(freq=freq*harmonic, dur=dur, amp=amp)
        if wave_type == 'triangle':
            sin = sin*(np.power(-1,i))*(harmonic**-2)
        else:
            sin = sin / harmonic
        wave = np.add(wave, sin)
        
    if wave_type == 'square':
        return wave * 4 / np.pi 
    if wave_type == 'triangle':
        return wave * np.pi / 4

In [ ]:
def group_close_indices(bin_indices):    
    bin_indices.sort()
    new_list = [bin_indices[0]]
    for i in range(1,len(bin_indices)):
        if(bin_indices[i] - 1 == bin_indices[i - 1]):
            if(isinstance(new_list[-1], list)):
                new_list[-1].append(bin_indices[i])
            else:
                new_list[-1] = [new_list[-1], bin_indices[i]]
        else:
            new_list.append(bin_indices[i])
    return new_list

def bin_to_freq_mag(indices, bin_to_freq, fftdata):
    output = []
    for idx in indices:
        if(isinstance(idx, list)):
            freqs = bin_to_freq[idx]
            mags = fftdata[idx]
            output.append((np.mean(freqs),np.mean(mags)))
        else:
            output.append((bin_to_freq[idx], fftdata[idx]))
    return output

def get_n_top_freqs(fftdata, N, bin_to_freq):
    """ return frequency, amplitude"""
    indices = np.argpartition(fftdata,-N)[-N:]
    grouped_indices = group_close_indices(indices)
    print(grouped_indices)
    return bin_to_freq_mag(grouped_indices, bin_to_freq, fftdata)

In [ ]:
def combine_freq(freqs, duration):
    output = oscillator(0, duration, amp=0)
    for freq in freqs:
        output = np.add(output, oscillator(freq[0], duration, amp=freq[1]))
    return output

In [ ]:
ipd.Audio(OPENA_WAV, rate=44100)

In [ ]:
def clean_input(audio):
    audio_mag = np.abs(audio)
    audio_mag = audio_mag[np.argmax(audio_mag>0.1):]
    b = audio_mag[::-1]
    i = len(b) - np.argmax(b > 0.1) - 1
    audio_mag = audio_mag[:i]
    return audio_mag
def normalize(audio_signal):
    audio_signal = audio_signal.astype(np.float32) / 32767.0 
    return (0.9 / max(audio_signal)) * audio_signal

In [ ]:
y, sr = librosa.load(OPENA_WAV)

In [ ]:
y = normalize(y)
y = clean_input(y)
# ipd.Audio(y, rate=sr)

In [ ]:
plt.figure()
plt.plot(y[:])
plt.show()

In [ ]:
stft = False
N_FFT = 2048
if(stft):
    frame_n = 5
    D = np.abs(librosa.stft(y, n_fft=N_FFT, hop_length=2048, win_length=2048))
    frame = D[:, frame_n]
else:
    D = np.abs(scipy.fft(y, n=N_FFT))
    frame = D[:len(D)//2]

In [ ]:
plt.figure()
plt.plot(frame[:10000])

In [ ]:
N = 10
bin_to_freq = librosa.fft_frequencies(sr, N_FFT)
top_N = get_n_top_freqs(frame, N, bin_to_freq)
top_N

In [ ]:
stft_test = combine_freq(top_N, 3)
ipd.Audio(stft_test, rate=44100)

### ...end of useful code

In [ ]:
plt.figure(dpi=800)

# D = librosa.amplitude_to_db(np.abs(librosa.stft(y, hop_length=2048, win_length=2048)), ref=np.max)
D = np.abs(librosa.stft(y, hop_length=2048, win_length=2048))
# plt.subplot(4, 2, 1)
specshow(D, y_axis='linear')
plt.colorbar(format='%+2.0f dB')
plt.title('Linear-frequency power spectrogram')

create an algorithm that extracts the N loudest frequencies and returns them

In [ ]:
plt.figure()
plt.plot(D[:,15])

In [ ]:
def pitch_fft(frame, srate): 
    mag = np.abs(np.fft.fft(frame))
    mag = mag[0:int(len(mag)/2)]
    plt.figure(figsize=(20,5))
    plt.plot(mag[:100])
    pitch_estimate = np.argmax(mag) * (srate / len(frame))
    return pitch_estimate 
hop_n = 10 * HOP_SIZE
pitch_fft(y[hop_n: hop_n + WIN_SIZE], sr)

# Take 2

In [ ]:
ipd.Audio(y, rate=44100)

In [ ]:
D2 = np.abs(scipy.fft(y, 2048))
D2 = D2[:len(D2)//2]

In [ ]:
plt.figure()
plt.plot(D2)

In [ ]:
d2_top = get_n_top_freqs(D2, 3)

In [ ]:
D2_test = combine_freq(d2_top, 3)
ipd.Audio(D2_test, rate=44100)

This is just me trying out some stuff. ~goh

In [ ]:
%matplotlib inline 
import math
import matplotlib.pyplot as plt
import numpy as np
import librosa as lb
import scipy.signal as signal 
from scipy.interpolate import interp1d 

import scipy.io.wavfile as wav
import IPython.display as ipd

import aubio

import timeit

In [ ]:
def pitch_track_aubio(signal, hopSize, winSize, extractor, srate=None): 
    offsets = np.arange(0, len(signal), hopSize)
    pitch_track = np.zeros(len(offsets))
    amp_track = np.zeros(len(offsets))
    
    for (m,o) in enumerate(offsets): 
        frame = signal[o:o+winSize] 
        amp_track[m] = np.sqrt(np.mean(np.square(frame)))  
        if(len(frame) < winSize):
            frame = np.pad(frame, (0,winSize - len(frame)), 'constant')
        extracted = extractor(frame)
#         print(extracted)
        pitch_track[m] = extracted[0]
        

        if (pitch_track[m] > 1500): 
            pitch_track[m] = 0 
        
    return (amp_track, pitch_track)

def sonify(amp_track, pitch_track, srate, hop_size):

    times = np.arange(0.0, float(hop_size * len(pitch_track)) / srate,
                      float(hop_size) / srate)

    # sample locations in time (seconds)                                                      
    sample_times = np.linspace(0, np.max(times), int(np.max(times)*srate-1))

    # create linear interpolators for frequencies and amplitudes                              
    # so that we have a frequency and amplitude value for 
    # every sample 
    freq_interpolator = interp1d(times,pitch_track)
    amp_interpolator = interp1d(times,amp_track)

    # use the interpolators to calculate per sample frequency and                             
    # ampitude values                                                                         
    sample_freqs = freq_interpolator(sample_times)
    sample_amps  = amp_interpolator(sample_times)

    # create audio signal                                                                     
    audio = np.zeros(len(sample_times));
    T = 1.0 / srate
    phase = 0.0
    
    # update phase according to the sample frequencies 
    for i in range(1, len(audio)):
        audio[i] = sample_amps[i] * np.sin(phase)
        phase = phase + (2*np.pi*T*sample_freqs[i])

    return audio

In [ ]:
VIBRATO_WAV = '../datasets/guitar/vibrato.wav'
OPENA_WAV = '../datasets/guitar/open-a.wav'
ipd.Audio(OPENA_WAV, rate=44100)

In [ ]:
audio, SR = lb.load(OPENA_WAV)

In [ ]:
hop_size = 256
win_size = 512
pitch_yin = aubio.pitch("yin",buf_size=win_size, hop_size=win_size, samplerate=SR)
pitch_yinfft = aubio.pitch("yinfft", buf_size=win_size, hop_size=win_size, samplerate=SR)

In [ ]:
(at1, pt1) = pitch_track_aubio(audio, hop_size, win_size, pitch_yin)
pt3 = signal.medfilt(pt1, kernel_size=7)

In [ ]:
pt1_audio = sonify(at1, pt3, SR, hop_size)
ipd.Audio(pt1_audio,rate=SR)